In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf

2022-11-24 20:49:07.709987: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, GRU, Dense, Flatten, Embedding, LSTM
from tensorflow.keras.models import Sequential

In [3]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
# stopwords removing
sw = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yasamanemami/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# from google.colab import drive
# drive.mount("/content/gdrive")

In [5]:
import pandas as pd
df = pd.read_csv('imdb_db.csv')

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df.shape


(50000, 2)

In [8]:
print(df["review"][3])

Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.


In [9]:
# Implement preprocessing function
import re

def clean(text):
    text = text.lower()
    text = re.sub("https?://\w+\.\w+\.\w+", " ", text)
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    result = []
    for word in text.split():
        if word not in sw:
            result.append(word)
    return " ".join(result)

In [10]:
X = df["review"].apply(clean)

In [11]:
X

0        one reviewers mentioned watching 1 oz episode ...
1        wonderful little production. <br /><br />the f...
2        thought wonderful way spend time hot summer we...
3        basically there's family little boy (jake) thi...
4        petter mattei's "love time money" visually stu...
                               ...                        
49995    thought movie right good job. creative origina...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    catholic taught parochial elementary schools n...
49998    i'm going disagree previous comment side malti...
49999    one expects star trek movies high art, fans ex...
Name: review, Length: 50000, dtype: object

In [12]:
y = np.where(df["sentiment"] == "positive", 1, 0)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [15]:
X_train

4109     film evil breed: legend samhain contains littl...
30907    film's tagline "you think know are. idea." rej...
23798    12th animated disney classic reasonable movie ...
12454    can't believe bad "film" is. starters, movie d...
34525    wanting see cut since day heard it, sometime l...
                               ...                        
40238    appears there's middle ground movie! takes pla...
1242     high hopes movie. theater monologue great nic ...
36375    stated countless times, "the hills eyes 2" rem...
47693    read 100 nancy drew books, bright enough catch...
29534    coltrane idle members bank robbery gang double...
Name: review, Length: 40000, dtype: object

In [16]:
X_train=np.array(X_train.values.tolist())
X_test=np.array(X_test.values.tolist())

In [17]:
from keras_preprocessing.text import Tokenizer

In [18]:
tokenizer = Tokenizer()

In [19]:
tokenizer.fit_on_texts(X_train)

In [20]:
word_index=tokenizer.word_index

In [21]:
word_index

{'br': 1,
 'movie': 2,
 'film': 3,
 'one': 4,
 'like': 5,
 'good': 6,
 'the': 7,
 'it': 8,
 'time': 9,
 'even': 10,
 'would': 11,
 'see': 12,
 'story': 13,
 'really': 14,
 'well': 15,
 'much': 16,
 'bad': 17,
 'get': 18,
 'great': 19,
 'also': 20,
 'people': 21,
 'first': 22,
 'made': 23,
 'make': 24,
 'way': 25,
 'movies': 26,
 'could': 27,
 'think': 28,
 'characters': 29,
 'watch': 30,
 'films': 31,
 'many': 32,
 'two': 33,
 'seen': 34,
 'character': 35,
 'i': 36,
 'never': 37,
 'love': 38,
 'plot': 39,
 'acting': 40,
 'life': 41,
 'best': 42,
 'know': 43,
 'little': 44,
 'show': 45,
 'this': 46,
 'ever': 47,
 'better': 48,
 'man': 49,
 'end': 50,
 'scene': 51,
 'still': 52,
 'and': 53,
 'say': 54,
 'scenes': 55,
 'something': 56,
 'go': 57,
 'back': 58,
 "i'm": 59,
 'real': 60,
 'watching': 61,
 'thing': 62,
 'actors': 63,
 'funny': 64,
 'old': 65,
 'though': 66,
 'years': 67,
 'another': 68,
 '10': 69,
 'work': 70,
 'actually': 71,
 'nothing': 72,
 'all': 73,
 'makes': 74,
 'direct

In [22]:
vocab_size = len(word_index)+1

In [23]:
print(vocab_size)

111479


In [24]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Keras Embedding Layer

In [25]:
maxlen = 100
X_train_pad = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test_pad = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [26]:
model = Sequential()
# all words in the vocub will have 30-dimensional vector (word embedding)
# you can initialize the weights of embedding parameters as we did for ANN/CNN
embedding_layer = Embedding(vocab_size, 30,
                            input_length=maxlen, 
                            embeddings_initializer='ones')

model.add(embedding_layer)

2022-11-24 20:49:52.415363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
#example: embedding the first review 
first_review_without_pad = X_train[0]
first_review_with_pad = X_train_pad[0]

In [28]:
first_review_without_pad

[3,
 386,
 5801,
 1885,
 21649,
 1333,
 44,
 101,
 712,
 12829,
 27734,
 3188,
 645,
 1210,
 312,
 234,
 984,
 2163,
 1314,
 234,
 40,
 1137,
 63,
 45,
 44,
 1144,
 2724,
 3,
 4795,
 25,
 904,
 21,
 2088,
 415,
 5,
 867,
 185,
 962,
 3278,
 1437,
 1058,
 101,
 14,
 1401,
 16,
 1531,
 5283,
 86,
 865,
 412,
 1532,
 5411,
 5888,
 234,
 378,
 243,
 1556,
 50915,
 22579,
 51,
 6367,
 364,
 137,
 1891,
 524,
 44,
 480,
 263,
 184,
 175,
 3,
 6,
 1063,
 210,
 1104,
 30,
 289,
 2552,
 2410,
 1598]

In [29]:
first_review_with_pad

array([    3,   386,  5801,  1885, 21649,  1333,    44,   101,   712,
       12829, 27734,  3188,   645,  1210,   312,   234,   984,  2163,
        1314,   234,    40,  1137,    63,    45,    44,  1144,  2724,
           3,  4795,    25,   904,    21,  2088,   415,     5,   867,
         185,   962,  3278,  1437,  1058,   101,    14,  1401,    16,
        1531,  5283,    86,   865,   412,  1532,  5411,  5888,   234,
         378,   243,  1556, 50915, 22579,    51,  6367,   364,   137,
        1891,   524,    44,   480,   263,   184,   175,     3,     6,
        1063,   210,  1104,    30,   289,  2552,  2410,  1598,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0], dtype=int32)

In [30]:
output_1st_review_without_pad = model.predict(first_review_without_pad)
output_1st_review_with_pad = model.predict(first_review_with_pad)

4/4 [==============================] - 0s 2ms/step


In [31]:
pd.DataFrame(output_1st_review_without_pad)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
76,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
77,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
78,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [32]:
# pd.DataFrame(output_1st_review_with_pad)

In [33]:
model.add(LSTM(units=32, dropout=0.7, recurrent_dropout=0.7))
model.add(Dense(1, activation='sigmoid'))

In [34]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc'])

In [35]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 30)           3344370   
                                                                 
 lstm (LSTM)                 (None, 32)                8064      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 3,352,467
Trainable params: 3,352,467
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
model.fit(X_train_pad, y_train, 
          batch_size=128, 
          epochs=20, 
          verbose=1, 
          validation_data=(X_test_pad, y_test))

Epoch 1/20
313/313 [==============================] - 43s 130ms/step - loss: 0.6972 - acc: 0.5019 - val_loss: 0.6941 - val_acc: 0.4914
Epoch 2/20
313/313 [==============================] - 39s 126ms/step - loss: 0.6948 - acc: 0.5001 - val_loss: 0.6931 - val_acc: 0.4914
Epoch 3/20
313/313 [==============================] - 40s 127ms/step - loss: 0.6936 - acc: 0.5047 - val_loss: 0.6924 - val_acc: 0.5454
Epoch 4/20
313/313 [==============================] - 39s 125ms/step - loss: 0.6928 - acc: 0.5130 - val_loss: 0.6912 - val_acc: 0.5532
Epoch 5/20
313/313 [==============================] - 40s 128ms/step - loss: 0.6908 - acc: 0.5266 - val_loss: 0.6848 - val_acc: 0.5966
Epoch 6/20
313/313 [==============================] - 39s 126ms/step - loss: 0.6797 - acc: 0.5652 - val_loss: 0.5898 - val_acc: 0.7524
Epoch 7/20
313/313 [==============================] - 39s 126ms/step - loss: 0.5764 - acc: 0.7056 - val_loss: 0.4351 - val_acc: 0.8088
Epoch 8/20
313/313 [==============================] - 3

KeyboardInterrupt: 

In [ ]:
word_embeddings = embedding_layer.get_weights()[0]

In [ ]:
pd.DataFrame(word_embeddings)

In [ ]:
# `word_to_index` is a mapping (i.e. dict) from words to their index, e.g. `love`: 69
embedding_4_each_words = {w:word_embeddings[idx] for w, idx in word_index.items()}

In [ ]:
embedding_4_each_words

In [ ]:
test = embedding_4_each_words['good']

In [ ]:
test = list(test)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

simil = cosine_similarity(word_embeddings, embedding_4_each_words['good'].reshape(1, -1))


In [ ]:
simil

In [ ]:
n = 5
top_n_similar = sorted(range(len(simil)), key=lambda i: simil[i])[-n-1:-1]


In [ ]:
for word, vector in embedding_4_each_words.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    for i in range (n):
        if np.array_equal(vector,word_embeddings[top_n_similar[i]]):
            print(word, simil[top_n_similar[i]])

In [ ]:
top_n_similar

In [ ]:
embedding_4_each_words["seasons"] == word_embeddings[2523]

In [ ]:
embedding_4_each_words["truer"] == word_embeddings[16734]

In [ ]:
embedding_4_each_words["blob"] == word_embeddings[8031]

# Word2Vec

In [ ]:
import gensim
import gensim.downloader as api

In [ ]:
# download the word2vec model
word2vec = api.load('word2vec-google-news-300')

In [ ]:
word2vec['boy']

In [ ]:
word2vec.most_similar("man")

In [ ]:
word2vec.most_similar("woman")

In [ ]:
word2vec.most_similar("king")

In [ ]:
word2vec.most_similar(positive= ["king", "woman"], negative= ['man'])